In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.optimizers import Adam

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy as cp

print(f'tf_version: {tf.__version__}')

2023-08-20 16:25:03.881589: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-20 16:25:04.220461: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-20 16:25:04.221453: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-20 16:25:06.922346: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


tf_version: 2.13.0


## Data Analysis

In [2]:
# Loading the dataset
original_df = pd.read_csv('./data/Housing.csv', sep=',')
df = cp.deepcopy(original_df)
display(df.head())

target = 'price'
features = [i for i in df.columns if i not in target]

x_train_df = df[['area', 'bedrooms', 'bathrooms', 'stories', 'parking']]
x_train_np = x_train_df.to_numpy()
y_train_np = df[target].to_numpy().reshape(-1,1)
print(f'x_train shape: {x_train_np.shape} \t y_train shape: {y_train_np.shape}')

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


x_train shape: (545, 5) 	 y_train shape: (545, 1)


The function implemented by a neuron with no activation is the same as linear regression:

![Alt text](images/linearfunction.png)

In [3]:
linear_layer = tf.keras.layers.Dense(units=1, activation='linear')
a1 = linear_layer(x_train_np[0].reshape(1,-1))
print(a1)
w, b = linear_layer.get_weights()
print(f'Initial parameters: \n{w}\n{b}')

tf.Tensor([[3479.622]], shape=(1, 1), dtype=float32)
Initial parameters: 
[[ 0.46904993]
 [ 0.0257802 ]
 [-0.11544752]
 [-0.17534876]
 [-0.03720117]]
[0.]


## Data Preprocessing

### Feature Scalling
In order to make the training algorithm faster, we normalize the inputs using Keras Normalization layer

In [4]:
print('Before Normalization:')
print(f'Peak-to-peak x_train: {np.ptp(x_train_np, axis=0)}')
display(pd.DataFrame(x_train_np, columns=x_train_df.columns).describe())

# Creating the normalization layer
norm_1 = tf.keras.layers.Normalization(axis=-1) 
# Does the same as passing mean and variance in layer construction
norm_1.adapt(x_train_np) 

# Foward propagating x_train
x_train_norm = norm_1(x_train_np)

print('\nAfter Normalization:')
print(f'Peak-to-peak x_train: {np.ptp(x_train_norm, axis=0)}')
display(pd.DataFrame(x_train_norm, columns=x_train_df.columns).describe())

Before Normalization:
Peak-to-peak x_train: [14550     5     3     3     3]


,area,bedrooms,bathrooms,stories,parking
count,545.000000,545.000000,545.000000,545.000000,545.000000
mean,5150.541284,2.965138,1.286239,1.805505,0.693578
std,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,6360.000000,3.000000,2.000000,2.000000,1.000000
max,16200.000000,6.000000,4.000000,4.000000,3.000000



After Normalization:
Peak-to-peak x_train: [6.7107935 6.7807055 5.9759965 3.4614208 3.4851508]


,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,5.450000e+02,5.450000e+02,5.450000e+02,5.450000e+02
mean,-8.399333e-08,-1.259900e-07,1.959844e-07,1.539878e-07,-2.799778e-08
std,1.000919e+00,1.000919e+00,1.000919e+00,1.000919e+00,1.000919e+00
min,-1.614530e+00,-2.665004e+00,-5.701866e-01,-9.293966e-01,-8.057413e-01
25%,-7.151452e-01,-1.308863e+00,-5.701866e-01,-9.293966e-01,-8.057413e-01
50%,-2.539223e-01,4.727817e-02,-5.701866e-01,2.244103e-01,-8.057413e-01
75%,5.578299e-01,4.727817e-02,1.421812e+00,2.244103e-01,3.559756e-01
max,5.096263e+00,4.115701e+00,5.405810e+00,2.532024e+00,2.679410e+00


### Tiling

Tile/copy our data to increase the training set size and reduce the number of training epochs.

In [5]:
x_train = np.tile(x_train_norm, (100, 1)) # Repeat 1000 times the rows while keeping the columns.
y_train = np.tile(y_train_np, (100, 1))

print(f'x_train shape before: {x_train_norm.shape}')
print(f'x_train shape after: {x_train.shape}')

x_train shape before: (545, 5)
x_train shape after: (54500, 5)


## Model Training

### Linear Model Training

In [37]:
tf.random.set_seed(100) # applied to achieve consistent results 

# Creates a linear neural network with 3 hidden layers 
L_0 = InputLayer(input_shape=(1,)) # This layer can be omitted 
L_1 = Dense(units=1, activation='linear', name='layer_1')

nn = Sequential([L_0, L_1])

display(nn.summary()) # The parameter counts correspond to the number of elements in the weight and bias.

W1, b1 = nn.get_layer("layer_1").get_weights()
print('Initialized weights: \n')
print(f"W1{W1.shape}:\n", W1, f"\nb1{b1.shape}:", b1, '\n')

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_1 (Dense)             (None, 1)                 2         
                                                                 
Total params: 2 (8.00 Byte)
Trainable params: 2 (8.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


None

Initialized weights: 

W1(1, 1):
 [[-0.17844772]] 
b1(1,): [0.] 



In [42]:
# Defines loss function and optimizer
nn.compile( 
    loss = 'mse',
    optimizer = Adam(learning_rate=10),
)

# Runs gradient descent and fits the weights to the data
nn.fit(
    x_train[:,0],y_train,            
    epochs=10,
)

Epoch 1/10
1704/1704 [==============================] - 4s 2ms/step - loss: 26084347215872.0000
Epoch 2/10
 842/1704 [=============>................] - ETA: 1s - loss: 25995876761600.0000

KeyboardInterrupt: 

### DNN Training

In [31]:
tf.random.set_seed(100)  # applied to achieve consistent results

# Creates a linear neural network with 3 hidden layers 
L_0 = InputLayer(input_shape=(x_train.shape[1],)) # This layer can be omitted 
L_1 = Dense(units=25, activation='sigmoid', name='layer_1')
L_2 = Dense(units=10, activation='sigmoid', name='layer_2')
L_3 = Dense(units=3, activation='sigmoid', name='layer_3')
L_4 = Dense(units=1, activation='linear', name='layer_4')

nn = Sequential([L_0, L_1, L_2, L_3, L_4])

display(nn.summary()) # The parameter counts correspond to the number of elements in the weight and bias.

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_1 (Dense)             (None, 25)                150       
                                                                 
 layer_2 (Dense)             (None, 10)                260       
                                                                 
 layer_3 (Dense)             (None, 3)                 33        
                                                                 
 layer_4 (Dense)             (None, 1)                 4         
                                                                 
Total params: 447 (1.75 KB)
Trainable params: 447 (1.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


None

In [25]:
# Defines loss function and optimizer
nn.compile( 
    loss = 'mean_squared_error',
    optimizer = Adam(learning_rate=0.1),
)

# Runs gradient descent and fits the weights to the data
nn.fit(
    x_train,y_train,            
    epochs=10,
)

Epoch 1/10
1704/1704 [==============================] - 3s 2ms/step - loss: 26212554506240.0000
Epoch 2/10
1704/1704 [==============================] - 3s 2ms/step - loss: 26210088255488.0000
Epoch 3/10
1704/1704 [==============================] - 3s 2ms/step - loss: 26207550701568.0000
Epoch 4/10
1704/1704 [==============================] - 2s 1ms/step - loss: 26205040410624.0000
Epoch 5/10
1704/1704 [==============================] - 3s 2ms/step - loss: 26202555285504.0000
Epoch 6/10
1704/1704 [==============================] - 3s 2ms/step - loss: 26200068063232.0000
Epoch 7/10
1704/1704 [==============================] - 3s 2ms/step - loss: 26197551480832.0000
Epoch 8/10
1704/1704 [==============================] - 3s 2ms/step - loss: 26195011829760.0000
Epoch 9/10
1704/1704 [==============================] - 3s 2ms/step - loss: 26192547676160.0000
Epoch 10/10
1704/1704 [==============================] - 3s 2ms/step - loss: 26190064648192.0000


In [16]:
p = nn.predict(x_train[:5,:])
print(p)
print('\n',y_train[:5])

1/1 [==============================] - 0s 26ms/step
[[ 7619949. ]
 [11286305. ]
 [ 7742543.5]
 [ 7475719. ]
 [ 5958961. ]]

 [[13300000]
 [12250000]
 [12250000]
 [12215000]
 [11410000]]
